In [1]:
import subprocess
import sys
import time
from itertools import zip_longest
import json,datetime,schedule

from trader import write_signal
from funcs import get_data,get_entrys_exits,get_entry_signals, validate_dfmpl

In [2]:
TZOFFSET=5*60*60 

## input arguments
param_choice = 15
#if len(sys.argv)>1:
#    param_choice = int(sys.argv[1])

trade_params = json.load(open("trade_params.json","r"))
trade_param = trade_params["ver2"]["params"][param_choice]
thres_diff = float(trade_param["thres_diff"])
tickerpair = trade_param["tickerpair"]
interval = trade_param["interval"]
sl = float(trade_param["sl"])
tp = float(trade_param["tp"])
percentile = float(trade_param["percentile"])

In [3]:
%%time
#dfmpl2 = get_data(tickerpair,interval,limit=100,type="live")
import pickle
# with open("test_dfmpldata.pkl","wb") as f:
#     pickle.dump(dfmpl2,f)
with open("test_dfmpldata.pkl","rb") as f:
    dfmpl2 = pickle.load(f)

CPU times: total: 0 ns
Wall time: 996 µs


In [67]:
for i in range(55,58):
    print(i)
    dfmpl=dfmpl2.iloc[:i]
    #dfmpl
    entrys,exits,_,_,_,_,_,_,thres_diff = get_entrys_exits(dfmpl,percentile,thres_diff)
    #print(len(entrys),len(exits),entrys,exits)
    print(entrys+1)
    print(exits+1)

55
[19 22]
[21 23]
56
[19 22]
[21 23]
57
[19 22 56]
[21 23]


In [69]:
dfmpl.iloc[-5:]

,Open,High,Low,Close,Volume
Date,,,,,
2023-07-16 11:40:00,27.85,27.90,27.77,27.80,15508.20
2023-07-16 11:45:00,27.80,27.86,27.79,27.82,8530.92
2023-07-16 11:50:00,27.81,27.83,27.74,27.77,11226.53
2023-07-16 11:55:00,27.76,27.86,27.75,27.85,12788.93
2023-07-16 12:00:00,27.85,28.18,27.85,28.08,85446.23


In [72]:
dfmpl.iloc[56]

Open         27.85
High         28.18
Low          27.85
Close        28.08
Volume    85446.23
Name: 2023-07-16 12:00:00, dtype: float64

In [6]:
#get all entry signal
entry_signals = get_entry_signals(entrys,dfmpl,onlybuy=True) 
new_entry=False
entry_df= None;buy=None
# todo , this will still report pervious entries as new entry on start
# todo redo run trades (possibly fixed)
for entry,exit in zip_longest(entrys,exits,fillvalue=None):
    if exit is None: # missing exit signal, so its a new enter signal
        buy_list = [ buy for ( entry_, buy, _ ) in entry_signals if entry_==entry]
        if len(buy_list)==1: 
            new_entry=True
            entry_df=dfmpl.iloc[entry]
            buy=buy_list[0]

In [74]:
hehe={"erth":234,"hert":341}
hehe2=hehe.copy()
hehe2["42r"]=2331
print(hehe,hehe2)

{'erth': 234, 'hert': 341} {'erth': 234, 'hert': 341, '42r': 2331}


In [76]:
eth234r="f342f"
eth234r2 = eth234r
eth234r+="#5tg34f"
print(eth234r,eth234r2)

f342f#5tg34f f342f


In [155]:
import datetime
from trader import price_action_signal
sl=-0.21
tp=0.1
dfname=""
hl_pairs=""
cur_price=100
ent_time=str(datetime.datetime.now())[:-4].replace(" ","_")
enter_data = {"price":cur_price,"sl":sl,"tp":tp,"dfname":dfname,"ent_time":ent_time,
            "hl_pairs":hl_pairs,"strat":"strat_tpsl1"}
strat_data = {"cur_sl":sl,"cur_tp":tp,"slip":-0.002,"ent_sl":sl,"ent_tp":tp,"strat":"strat_tpsl1"}

stdmean_status="HOLD"
stdmean_status_exited=False
pas_status="HOLD"
pas_strr=""


In [156]:
#while status=="HOLD":
if True:
    # get current price
    cur_price = 104#get_current_price(symbol)
    # get price_action_signal
    pas_status,strat_data,pas_strr = price_action_signal(enter_data,strat_data,cur_price)
    updated=False;update_list=[]
    print("**start",pas_strr)
    print("")
    update_list.append(pas_strr[:6])
    while (pas_strr[:2]=="Up"):
        # repeat until its not up anymore, then take the prev sl and tp
        prev_strat_data = strat_data.copy()
        prev_pas_status = pas_status
        prev_pas_strr = pas_strr
        pas_status,strat_data,pas_strr = price_action_signal(enter_data,strat_data,cur_price)
        if pas_strr[:2]=="Up":
            updated=True
            update_list.append(pas_strr[:6])
            print("**update",pas_strr)
    
    if updated:
        pas_status,strat_data,pas_strr = prev_pas_status,prev_strat_data.copy(),prev_pas_strr
        pas_strr = " ".join(update_list[:-1])+" "+pas_strr
    print("")
    print("**final",pas_strr)

**start UpMedi LP `85.2000`,`125.5000` (`-14.80`,`25.50`)%
Next levels: `94.5000`,`103.8000`,`113.1000`(`-5.50`,`3.80`,`13.10`)%, width=31.00%

**update UpMedi LP `91.4000`,`141.0000` (`-8.60`,`41.00`)%
Next levels: `100.7000`,`110.0000`,`119.3000`(`0.70`,`10.00`,`19.30`)%, width=31.00%
**update UpSlow LP `94.5000`,`144.1000` (`-5.50`,`44.10`)%
Next levels: `103.8000`,`113.1000`,`122.4000`(`3.80`,`13.10`,`22.40`)%, width=31.00%
**update UpSlow LP `97.6000`,`147.2000` (`-2.40`,`47.20`)%
Next levels: `106.9000`,`116.2000`,`125.5000`(`6.90`,`16.20`,`25.50`)%, width=31.00%

**final UpMedi UpMedi UpSlow UpSlow LP `97.6000`,`147.2000` (`-2.40`,`47.20`)%
Next levels: `106.9000`,`116.2000`,`125.5000`(`6.90`,`16.20`,`25.50`)%, width=31.00%


In [157]:
#while status=="HOLD":
if True:
    # get current price
    cur_price = 98#get_current_price(symbol)
    # get price_action_signal
    pas_status,strat_data,pas_strr = price_action_signal(enter_data,strat_data,cur_price)
    updated=False;update_list=[]
    print("**start",pas_strr)
    print("")
    update_list.append(pas_strr[:6])
    while (pas_strr[:2]=="Up"):
        # repeat until its not up anymore, then take the prev sl and tp
        prev_strat_data = strat_data.copy()
        prev_pas_status = pas_status
        prev_pas_strr = pas_strr
        pas_status,strat_data,pas_strr = price_action_signal(enter_data,strat_data,cur_price)
        if pas_strr[:2]=="Up":
            updated=True
            update_list.append(pas_strr[:6])
            print("**update",pas_strr)
    
    if updated:
        pas_status,strat_data,pas_strr = prev_pas_status,prev_strat_data.copy(),prev_pas_strr
        pas_strr = " ".join(update_list[:-1])+" "+pas_strr
    print("")
    print("**final",pas_strr)

**start In SLTP LP `97.6000`,`147.2000` (`-2.40`,`47.20`)%
Next levels: `106.9000`,`116.2000`,`125.5000`(`6.90`,`16.20`,`25.50`)%, width=31.00%


**final In SLTP LP `97.6000`,`147.2000` (`-2.40`,`47.20`)%
Next levels: `106.9000`,`116.2000`,`125.5000`(`6.90`,`16.20`,`25.50`)%, width=31.00%


In [158]:
#while status=="HOLD":
if True:
    # get current price
    cur_price = 125#get_current_price(symbol)
    # get price_action_signal
    pas_status,strat_data,pas_strr = price_action_signal(enter_data,strat_data,cur_price)
    updated=False;update_list=[]
    print("**start",pas_strr)
    print("")
    update_list.append(pas_strr[:6])
    while (pas_strr[:2]=="Up"):
        # repeat until its not up anymore, then take the prev sl and tp
        prev_strat_data = strat_data.copy()
        prev_pas_status = pas_status
        prev_pas_strr = pas_strr
        pas_status,strat_data,pas_strr = price_action_signal(enter_data,strat_data,cur_price)
        if pas_strr[:2]=="Up":
            updated=True
            update_list.append(pas_strr[:6])
            print("**update",pas_strr)
    
    if updated:
        pas_status,strat_data,pas_strr = prev_pas_status,prev_strat_data.copy(),prev_pas_strr
        pas_strr = " ".join(update_list[:-1])+" "+pas_strr
    print("")
    print("**final",pas_strr)

**start UpMedi LP `103.8000`,`162.7000` (`3.80`,`62.70`)%
Next levels: `113.1000`,`122.4000`,`131.7000`(`13.10`,`22.40`,`31.70`)%, width=31.00%

**update UpMedi LP `110.0000`,`178.2000` (`10.00`,`78.20`)%
Next levels: `119.3000`,`128.6000`,`137.9000`(`19.30`,`28.60`,`37.90`)%, width=31.00%
**update UpSlow LP `113.1000`,`181.3000` (`13.10`,`81.30`)%
Next levels: `122.4000`,`131.7000`,`141.0000`(`22.40`,`31.70`,`41.00`)%, width=31.00%
**update UpSlow LP `116.2000`,`184.4000` (`16.20`,`84.40`)%
Next levels: `125.5000`,`134.8000`,`144.1000`(`25.50`,`34.80`,`44.10`)%, width=31.00%

**final UpMedi UpMedi UpSlow UpSlow LP `116.2000`,`184.4000` (`16.20`,`84.40`)%
Next levels: `125.5000`,`134.8000`,`144.1000`(`25.50`,`34.80`,`44.10`)%, width=31.00%


In [159]:
#while status=="HOLD":
if True:
    # get current price
    cur_price = 144#get_current_price(symbol)
    # get price_action_signal
    pas_status,strat_data,pas_strr = price_action_signal(enter_data,strat_data,cur_price)
    updated=False;update_list=[]
    print("**start",pas_strr)
    print("")
    update_list.append(pas_strr[:6])
    while (pas_strr[:2]=="Up"):
        # repeat until its not up anymore, then take the prev sl and tp
        prev_strat_data = strat_data.copy()
        prev_pas_status = pas_status
        prev_pas_strr = pas_strr
        pas_status,strat_data,pas_strr = price_action_signal(enter_data,strat_data,cur_price)
        if pas_strr[:2]=="Up":
            updated=True
            update_list.append(pas_strr[:6])
            print("**update",pas_strr)
    
    if updated:
        pas_status,strat_data,pas_strr = prev_pas_status,prev_strat_data.copy(),prev_pas_strr
        pas_strr = " ".join(update_list[:-1])+" "+pas_strr
    print("")
    print("**final",pas_strr)

**start UpMedi LP `122.4000`,`199.9000` (`22.40`,`99.90`)%
Next levels: `131.7000`,`141.0000`,`150.3000`(`31.70`,`41.00`,`50.30`)%, width=31.00%

**update UpMedi LP `128.6000`,`215.4000` (`28.60`,`115.40`)%
Next levels: `137.9000`,`147.2000`,`156.5000`(`37.90`,`47.20`,`56.50`)%, width=31.00%
**update UpSlow LP `131.7000`,`218.5000` (`31.70`,`118.50`)%
Next levels: `141.0000`,`150.3000`,`159.6000`(`41.00`,`50.30`,`59.60`)%, width=31.00%
**update UpSlow LP `134.8000`,`221.6000` (`34.80`,`121.60`)%
Next levels: `144.1000`,`153.4000`,`162.7000`(`44.10`,`53.40`,`62.70`)%, width=31.00%

**final UpMedi UpMedi UpSlow UpSlow LP `134.8000`,`221.6000` (`34.80`,`121.60`)%
Next levels: `144.1000`,`153.4000`,`162.7000`(`44.10`,`53.40`,`62.70`)%, width=31.00%


In [160]:
#while status=="HOLD":
if True:
    # get current price
    cur_price = 136#get_current_price(symbol)
    # get price_action_signal
    pas_status,strat_data,pas_strr = price_action_signal(enter_data,strat_data,cur_price)
    updated=False;update_list=[]
    print("**start",pas_strr)
    print("")
    update_list.append(pas_strr[:6])
    while (pas_strr[:2]=="Up"):
        # repeat until its not up anymore, then take the prev sl and tp
        prev_strat_data = strat_data.copy()
        prev_pas_status = pas_status
        prev_pas_strr = pas_strr
        pas_status,strat_data,pas_strr = price_action_signal(enter_data,strat_data,cur_price)
        if pas_strr[:2]=="Up":
            updated=True
            update_list.append(pas_strr[:6])
            print("**update",pas_strr)
    
    if updated:
        pas_status,strat_data,pas_strr = prev_pas_status,prev_strat_data.copy(),prev_pas_strr
        pas_strr = " ".join(update_list[:-1])+" "+pas_strr
    print("")
    print("**final",pas_strr)

**start In SLTP LP `134.8000`,`221.6000` (`34.80`,`121.60`)%
Next levels: `144.1000`,`153.4000`,`162.7000`(`44.10`,`53.40`,`62.70`)%, width=31.00%


**final In SLTP LP `134.8000`,`221.6000` (`34.80`,`121.60`)%
Next levels: `144.1000`,`153.4000`,`162.7000`(`44.10`,`53.40`,`62.70`)%, width=31.00%
